In [1]:
import numpy as np
import meshplot as mp
import meshio
import triangle

In [2]:
def generate_isotropic_square(n=50, allow_steiner=True):
    points = []
    for x in np.linspace(0, 1, n):
        for y in [0, 1]:
            points.append([x, y])
    for y in np.linspace(0, 1, n):
        for x in [0, 1]:
            points.append([x, y])
    points = np.array(points) - [0.5, 0.5] # center the square
    max_area = 1 / n**2
    tri = triangle.triangulate({"vertices": points}, f'{"" if allow_steiner else "Y"}a{max_area:f}q')
    return meshio.Mesh(tri["vertices"], [("triangle", tri["triangles"])])

In [3]:
def generate_anisotropic_square(nx=50, p=2, min_ny=4, max_ny=None, allow_steiner=True):
    points = []
    if max_ny is None:
        max_ny = nx
    for x in np.linspace(0, 1, nx):
        ny = int((max_ny - min_ny) * (1 - x) + min_ny)
        for y in np.linspace(0, 1, ny):
            points.append([x**p, y])
    points = np.array(points) - [0.5, 0.5] # center the square

    tri = triangle.triangulate({"vertices": points}, f'{"" if allow_steiner else "Y"}q')
    return meshio.Mesh(tri["vertices"], [("triangle", tri["triangles"])])

In [4]:
def plot_mesh(mesh, **kwargs):
    shading = {
        "flat": True, # Flat or smooth shading of triangles
        "wireframe": True, "wire_width": 0.1, "wire_color": "black", # Wireframe rendering
        "width": 600, "height": 600, # Size of the viewer canvas
        "antialias": True, # Antialising, might not work on all GPUs
        "scale": 2.0, # Scaling of the model
        "side": "DoubleSide", # FrontSide, BackSide or DoubleSide rendering of the triangles
        "colormap": "viridis", "normalize": [None, None], # Colormap and normalization for colors
        "background": "#ffffff", # Background color of the canvas
        "line_width": 2.0, "line_color": "black", # Line properties of overlay lines
        "bbox": False, # Enable plotting of bounding box
        "point_color": "red", "point_size": 0.05 # Point properties of overlay points
    }
    shading.update(kwargs)
    return mp.plot(mesh.points, mesh.cells[0].data, shading=shading)

In [5]:
for n in [2, 11, 21, 31, 41, 51]:
    n = int(n)
    isotropic_square = generate_isotropic_square(n=n)
    isotropic_square.write(f"../meshes/square/isotropic_h={1/(n-1):g}.msh", file_format="gmsh")

In [6]:
# plot_mesh(isotropic_square)

/usr/local/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [ ]:
anisotropic_square = generate_anisotropic_square(max_ny=100)
anisotropic_square.write("../meshes/square/anisotropic.msh", file_format="gmsh")
# plot_mesh(anisotropic_square)